#### Chain Of Thoughts With RAG
What is Chain-of-Thought (CoT) in RAG?

CoT reasoning breaks down a complex question into intermediate steps, and allows retrieval + reflection at each step before answering.

User Query
   ↓
- Step 1: Decompose question → sub-steps (Reason)
- Step 2: Retrieve docs per step (Act)
- Step 3: Combine context (Observe)
- Step 4: Final answer generation (Reflect)

In [2]:
import os
from typing import List
from pydantic import BaseModel

from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langgraph.graph import StateGraph, START, END

In [3]:
## 1. Prepare Vector store

docs = TextLoader("research_notes.txt",encoding="utf-8").load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500, 
    chunk_overlap = 50
)

chunks = splitter.split_documents(docs)
embedding = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever()

In [4]:
import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
load_dotenv()


llm = init_chat_model("openai:gpt-4o")


In [5]:
## 2. Langgraph State Definition

class RAGCoTState(BaseModel):
    question: str
    sub_steps: list[str] = []
    retrieved_docs: List[Document] = []
    answer:str = ""

In [17]:
## 3. Nodes

## a. Plan Sub Question

def plan_steps(state: RAGCoTState) -> RAGCoTState:
    prompt = f"Break the question into 2-3 reasoning steps: \n\n {state.question}"
    result = llm.invoke(prompt).content
    subqs = [line.strip("- ")for line in result.split("\n") if line.strip()]

    return state.model_copy(update={"sub_steps": subqs})

In [18]:
## b. Retrievr for each step

def retrieve_per_step(state: RAGCoTState) -> RAGCoTState:
    all_docs = []
    for sub in state.sub_steps:
        docs = retriever.invoke(sub)
        all_docs.extend(docs)
    
    return state.model_copy(update={"retrieved_docs": all_docs})

In [19]:
## c. Generate Final Answer

def generate_answer(state:RAGCoTState) -> RAGCoTState:
    context = "\n\n".join([doc.page_content for doc in state.retrieved_docs])
    prompt = f"""
        You are answering a complex question using reasoning and retrieved documents.

        Question: {state.question}

        Relevant Information:
        {context}

        Now synthesize a well-reasoned final answer.
        """
    result = llm.invoke(prompt).content.strip()
    return state.model_copy(update = {"answer": result})

In [20]:
### 4. Langgraph Graph

builder = StateGraph(RAGCoTState)

builder.add_node("planner", plan_steps)
builder.add_node("retriever", retrieve_per_step)
builder.add_node("responder", generate_answer)

builder.set_entry_point("planner")
builder.add_edge("planner", "retriever")
builder.add_edge("retriever", "responder")
builder.add_edge("responder", END)

graph = builder.compile()

In [21]:
graph

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 400.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [22]:
# -------------------------------
# 5. Run CoT RAG Agent
# -------------------------------
if __name__ == "__main__":
    query = "what are the additional eperiments in Transformer eveluation?"
    state = RAGCoTState(question=query)
    final = graph.invoke(state)

    print("\n🪜 Reasoning Steps:", final["sub_steps"])
    print("\n✅ Final Answer:\n", final["answer"])


🪜 Reasoning Steps: ['To address the question of what additional experiments are involved in evaluating Transformers, we can break it down into the following reasoning steps:', '1. **Identify Core Evaluation Metrics and Benchmarks:**', 'Begin by understanding the standard evaluation metrics typically used for Transformers, such as accuracy, F1 score, BLEU score (for translation tasks), etc.', 'Consider the benchmark datasets typically used to evaluate Transformer models in different tasks like NLP, vision, etc., for example, GLUE, SuperGLUE, ImageNet, etc.', '2. **Determine Additional Experimental Dimensions:**', 'Assess the need for additional evaluation dimensions such as robustness (adversarial testing, noise), model interpretability, and efficiency (inference time, computational resource usage).', 'Consider experiments focusing on ethical aspects like bias detection, fairness, and the impact of model compressions or optimizations on performance.', '3. **Explore Advanced Evaluation 